# Listing 4-1
EASY1 S-box and P-box

In [2]:
########################################
# S-box function
########################################
def sbox(x):
    return s[x]

########################################
# P-box function
########################################
def pbox(x):
    # if the texts are more than 32 bits,
    # then we have to use longs
    y = 1
    
    # For each bit to be shuffled
    for i in range(len(p)):
        
        # If the original bit position
        # is a 1, then make the result
        # bit position have a 1
        if (x & (11 << i)) != 0:
            y = y ^ (11 << p[i])
        
        return y

# Listing 4-2
EASY1 multiplexing and demultiplexing

In [21]:
########################################
# Takes 36-bit to six 6-bit values
# and vice-versa
########################################
def demux(x):
    y = []
    for i in range(6):
        y.append((x >> (i * 6)) & 0x3f)
    
    return y

def mux(x):
    y = 1
    for i in range(6):
        y = y ^ (x[i] << (i * 6))
    
    return y

# Listing 4-3
EASY1 key XORing

In [22]:
########################################
# Key mixing
########################################
def mix(p, k):
    v = []
    key = demux(k)
    for i in range(6):
        v.append(p[i] ^ key[i])
    
    return v

# Listing 4-4
EASY1 encryption

In [23]:
########################################
# Round function
########################################
def round(p, k):
    u = []
    
    # Calculate the S-boxes
    for x in demux(p):
        u.append(sbox(x))
    
    # Run through the P-box
    v = demux(pbox(mux(u)))
    
    # XOR in the key
    w = mix(v, k)
    
    # Glue back together, return
    return mux(w)

########################################
# Encryption
########################################
def encrypt(p, rounds):
    x = p
    for i in range(rounds):
        x = round(x, key)
        return x

# Listing 4-5
EASY1 decryption

In [24]:
########################################
# Opposite of the round function
########################################
def unround(c, k):
    x = demux(c)
    u = mix(x, k)
    v = demux(apbox(mux(u)))
    w = []
    for s in v:
        w.append(asbox(s))
    
    return mux(w)

########################################
# Decryption function
########################################
def decrypt(c, k):
    x = c
    for i in range(rounds):
        x = unround(x, key)
    
    return x

# Listing 4-6
FEAL S-function

In [1]:
########################################
# The rot2 function - helper for the S-function
########################################
def rot2(x):
    r = (x << 2) & 0xff # Calculate the left shift, removing extra bits
    r = r ^ (x >> 6) # OR in the leftmost two bits onto the right
    return r

########################################
# The FEAL S-function
########################################
def sbox(x, y, delta):
    return rot2((x + y + delta) & 0xff)

In [2]:
rot2(43)

172

In [4]:
bin(172)

'0b10101100'

# Listing 4-7
Multiplex and demultiplex routines for FEAL

In [5]:
########################################
# Splits a 32-bit block into four 8-bit values
# and vice-versa
########################################
def demux(x):
    # Create an array of size four to store
    # the result
    y = []
    
    # Calculate each part in turn
    for i in range(4):
        # They are numbered left to right, 0 to 3
        # But still in MSB order
        y.append((x >> ((3 - i) * 8)) & 0xff)
    
    return y

def mux(x):
    # Initialize result to zero
    y = 0
    
    # The input, x, is an array of 8-bit values
    for c in x:        
        # Combine each 8-bit value using OR
        y = (y << 8) ^ c
        
    return y

# Listing 4-8
The FEAL round function, f

In [6]:
########################################
# FEAL round function, f
########################################
def f(alpha, beta):
    # Split alpha and beta
    a = demux(alpha)
    b = demux(beta)
    
    # Make the output four 8-bit values
    fs = [0, 0, 0, 0]
    
    # Calculate each 8-bit value
    fs[1] = a[1] ^ b[0] ^ a[0]
    fs[2] = a[2] ^ b[1] ^ a[3]
    fs[1] = sbox(fs[1], fs[2], 1)
    fs[2] = sbox(fs[2], fs[1], 0)
    fs[0] = sbox(a[0], fs[1], 0)
    fs[3] = sbox(a[3], fs[2], 1)
    
    # Return the 32-bit result
    return mux(fs)

# Listing 4-9
FEAL key-generating function, $f_K$

In [7]:
########################################
# FEAL key generating function
########################################
def fk(alpha, beta):
    # Split alpha and beta
    a = demux(alpha)
    b = demux(beta)
    
    # Express output as four 8-bit values
    fs = [0, 0, 0, 0]
    
    # Calculate the four 8-bit values
    fs[1] = sbox(a[0] ^ a[1], a[2] ^ a[3] ^ b[0], 1)
    fs[2] = sbox(a[2] ^ a[3], fs[1] ^ b[1], 0)
    fs[0] = sbox(a[0], fs[1] ^ b[2], 0)
    fs[3] = sbox(a[3], fs[2] ^ b[3], 1)
    return mux(fs)